Import the libraries :

In [ ]:
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.color import rgb2lab, lab2rgb, gray2rgb
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf

Import  pretrained VGG16 model

In [ ]:
vgg16 = VGG16()
vgg_up_to_19th = Sequential() 

Take the encoder part (first 19 layers) of VGG16 model and add the layes into our sequential model

In [ ]:
for i, layer in enumerate(vgg16.layers):
    if i<19:          
      vgg_up_to_19th.add(layer)
vgg_up_to_19th.summary()

we want the layers of VGG16 with its original weights without changing them,so that we set the trainable parameter in each layer to false because we don’t want to train them again

In [ ]:
for layer in vgg_up_to_19th.layers:
  layer.trainable=False

Initialize the path of the training images folder

In [ ]:
path = '/content/drive/MyDrive/project work/human dataset'

Normalize the images - divide by 255

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

VGG16 is expecting an image of 3 dimension with size 224x224 as an input, 
in preprocessing we have to scale all images to 224

In [ ]:
train = train_datagen.flow_from_directory(path, target_size=(224, 224), batch_size=1500,class_mode=None)

By iterating on each image, we convert the RGB to Lab,where L channel repressents lightness and a,b channel represent the color of the images. So,we can consider the L channel as greyscale image and a,b channel as the colorized version of it.

In [ ]:
X =[]
Y =[]

for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0])
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128,
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
    print('error')

Convert X and Y into numpy arrays and change their dimensions

In [ ]:
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))

The L channel of the image has only 1 dimension but vgg16 needs 3 dimensions to work .So we are converting the L channel grey image to rgb to add extra 2 layers.
Then We are passing that image to our vgg16 model for prediction.And the prediction result will be used as a input to the decoder which is created by us.

In [ ]:
vggfeatures = []
for i, sample in enumerate(X):
  sample = gray2rgb(sample)
  sample = sample.reshape((1,224,224,3))
  prediction = vgg_up_to_19th.predict(sample)
  prediction = prediction.reshape((7,7,512))
  vggfeatures.append(prediction)
vggfeatures = np.array(vggfeatures)

Adding decoding layers to our model

In [ ]:
model = Sequential()

model.add(Conv2D(256, (3,3), activation='relu', padding='same', input_shape=(7,7,512)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.summary()

Train the model and save the model

In [ ]:
tf_callbacks=tf.keras.callbacks.TensorBoard(log_dir='logs/',histogram_freq=1)

model.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])
model.fit(vggfeatures, Y, verbose=1, epochs=1000, batch_size=40,callbacks=[tf_callbacks])
model.save('human_model.model',save_format='h5')